In [5]:
from selenium.webdriver.common.by import By
import time
from tqdm.autonotebook import tqdm
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType

driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))

In [17]:
driver.get("https://www.finam.ru/quote/moex-akcii/mmk/publications/date/2023-06-01/2023-07-31/")

In [18]:
while (
    driver.find_element(
        By.XPATH, '//*[@id="finfin-local-plugin-block-item-publication-list-filter-date-showMore-container"]'
    ).value_of_css_property("display")
    != "none"
):
    button = driver.find_element(
        By.CSS_SELECTOR, "#finfin-local-plugin-block-item-publication-list-filter-date-showMore-container > span"
    )
    try:
        button.click()
    except Exception as e:
        print(e)
    time.sleep(1)

In [19]:
links = []
for elem in driver.find_elements(
    By.CSS_SELECTOR, "#finfin-local-plugin-block-item-publication-list-filter-date-content > a"
):
    links.append(elem.get_attribute("href"))

In [20]:
texts = []
for link in tqdm(links):
    driver.get(link)
    if link.find("www.finam.ru") != -1:
        texts_html = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[4]/div[1]/div[3]/div[2]")
    else:
        texts_html = driver.find_elements(By.CSS_SELECTOR, "#content-block > dl:nth-child(4) > dd:nth-child(1) > p")
    text = " ".join(elem.text for elem in texts_html)
    texts.append({"url": link, "text": text})
    time.sleep(1)

100%|██████████| 28/28 [03:12<00:00,  6.88s/it]


In [21]:
pd.DataFrame(texts).to_csv("mmk_news.csv")

In [23]:
gazprom = pd.read_csv("gazprom_news.csv", index_col=0)
gazprom["company"] = "gazprom"
mmk = pd.read_csv("mmk_news.csv", index_col=0)
mmk["company"] = "mmk"
news = pd.concat([gazprom, mmk])
news.shape

(151, 3)

In [24]:
news.to_csv("news.csv.zip")